# Analyse
---
Dieses Modul ist zuständig für die Erstellung von Grafiken zur Analyse der in den Simulationen erzeugten Daten.

#### Funktionsweise

In diesem Modul werden die folgenden Funktionen verwendet:

1. **Erzeugung von Grafiken**: Es werden verschiedene Diagramme generiert, um verschiedene Aspekte der Simulationsergebnisse zu visualisieren. Diese Diagramme umfassen Verteilungen von Energieverbrauch und -produktion, Entwicklung der Strompreise im Großhandelsmarkt, Verteilung der Stromkosten usw.

2. **Erstellung von Berichten**: Es kann ein vollständiger Bericht über die Simulation generiert werden, der die generierten Diagramme und andere relevante Daten enthält.

3. **Daten herunterladen**: Es wird die Möglichkeit geboten, die Simulationsdaten im Excel-Format herunterzuladen, um eine detailliertere Analyse außerhalb der Anwendung durchzuführen.

#### Quellcode

In [2]:
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html, Input, Output, State
import base64
import plotly.graph_objects as go
import os
import pandas as pd
import calendar
import plotly.express as px

data = None
predictions = None

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, 'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css'])

##############################################################################################################
# Layout
##############################################################################################################
def build_layout():

    accordion_item = dbc.AccordionItem(
        children=readme_blocks(),
        title="Smart Grid Strategist",
    )
    
    return dbc.Container(
        fluid=True,
        children=[
            dbc.Row(
                dbc.Col(
                    dbc.Accordion(
                        [
                            accordion_item
                        ],
                        start_collapsed=True,
                    ),
                    style={'margin-bottom': '10px'}
                )
            ),
            dbc.Row(
                dbc.Col(
                    dbc.Accordion(
                        [
                            dbc.AccordionItem(
                                [
                                    dbc.Row(
                                        [
                                            dbc.Col(
                                                [
                                                    html.Label("Kategorie des Letztverbrauchers:", style={'margin-bottom': '10px'}),                                                    
                                                    dbc.RadioItems(id='user-type-radio',
                                                                   options=[
                                                                       {'label': 'Consumer', 'value': 'consumer'},
                                                                       {'label': 'Prosumer', 'value': 'prosumer'},
                                                                       {'label': 'Flexumer', 'value': 'flexumer'}
                                                                   ],
                                                                   value='consumer',
                                                                   inline=True,
                                                                   style={'margin-bottom': '10px'}
                                                                  ),
                                                    html.Img(id='dynamic-image', style={'width': '100%', 'height': 'auto', 'margin-bottom': '10px'}),
                                                          
                                                ],
                                                width=6
                                            ),
                                            dbc.Col(
                                                [
                                                    html.Label("Jährlicher Stromverbrauch (kWh):"),
                                                    dbc.Input(id='annual-consumption-input', type='number', step=1, value=2500, style={'margin-bottom': '10px'}),
                                                    html.Label("Aktueller Stromtarif (Ct./kWh):"),
                                                    dbc.Input(id='current-tariff-input', type='number', step=0.01, value=34.37, style={'margin-bottom': '10px'}),
                                                    html.Label("Grundpreis (€/Monat):"),
                                                    dbc.Input(id='current-base-input', type='number', step=0.01, value=14.96, style={'margin-bottom': '10px'}),
                                                    html.Label("Stromerzeugungskapazität der PV-Anlage:", style={'margin-bottom': '10px'}),
                                                    dbc.RadioItems(id='pv-capacity', inline=True, style={'margin-bottom': '10px'}),
                                                    html.Label("Stromspeicherkapazität:", style={'margin-bottom': '10px'}),
                                                    dbc.RadioItems(id='battery-capacity', inline=True, style={'margin-bottom': '10px'})                                    
                                                ],
                                                width=6
                                            )
                                        ]
                                    ),
                                    dbc.Row([dbc.Col(dbc.Button('Simulieren', id='simulate-button', n_clicks=0, color="primary", className="mt-3", style={'width': '100%'}))])
                                ],
                                 title="Simulator"
                            )
                        ]
                    ),
                    style={'margin-bottom': '10px'}
                )
            ),
            dbc.Row(
                dbc.Col(
                    dbc.Accordion(
                        [
                            dbc.AccordionItem(
                                [
                                    dbc.Row(
                                        [
                                            dbc.Col(
                                                dcc.DatePickerRange(display_format='DD/MM/YYYY'),  # TODO
                                                width=4
                                            ),
                                            dbc.Col(
                                                dbc.Button('Simulationsbericht generieren', id='generate-report-button', color='secondary'),  # TODO
                                                width=4
                                            ),
                                            dbc.Col(
                                                dbc.Button(id='save-button',
                                                           children=[
                                                                html.I(className="fas fa-download mr-2"),
                                                               'Simulationsdaten herunterladen'
                                                           ],
                                                           color='secondary',
                                                          ),
                                                width=4
                                            )
                                        ]
                                    ),
                                    dbc.Row(
                                        dbc.Col(
                                            dbc.CardBody(
                                                dbc.Spinner(html.Div(id='simulation-results'))
                                            )
                                        )
                                    )
                                ],
                                title="Analyse"
                            )
                        ],
                        id='simulation-view',
                        style={'display': 'none'},
                        
                    )
                )
            )
        ],
         style={"background-color": "#333", "height": "100%"}
    )

##############################################################################################################
# Helper Methods
##############################################################################################################
def readme_blocks():
    return [
        html.P("Willkommen bei Smart Grid Strategist!",style = {'margin': '2', 'font-weight': 'bold'})      
    ]

def load_image(user_type):
    base_dir = 'SGS/Bilder/'
    if user_type == 'consumer':
        image_filename = 'Consumer.png'
    elif user_type == 'prosumer':
        image_filename = 'Prosumer.png'
    elif user_type == 'flexumer':
        image_filename = 'Flexumer.png'
    else:
        image_filename = 'Consumer.png'

    with open(base_dir + image_filename, 'rb') as f:
        image = f.read()
    encoded_image = base64.b64encode(image).decode()

    return f"data:image/png;base64,{encoded_image}"

def generate_graphs(user_type):
    graphs = []
    graph_creators = [
        create_graph_1,
        create_graph_2,
        create_graph_3,
        create_graph_4,
        create_graph_5,
        create_graph_6,
        create_graph_7,
        create_graph_8,
        create_graph_9
    ]

    for i, create_graph in enumerate(graph_creators, start=1):
        figure = create_graph(user_type) if i <= 4 else create_graph()
        graphs.extend([html.Hr(), dcc.Graph(id=f'graph-{i}', figure=figure)])
    
    return graphs

def save_to_excel():
    global data, predictions
    try:
        output_folder = 'SGS/Output-Data'
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        excel_file_1_path = os.path.join(output_folder, 'Simulationsergebnis.xlsx')
        excel_file_2_path = os.path.join(output_folder, 'Vorhersagen_2024.xlsx')
        data.to_excel(excel_file_1_path, index=False)
        predictions.to_excel(excel_file_2_path, index=False)
    except Exception as e:
        print(f"Fehler beim Speichern von DataFrame in Excel: {e}")
        
##############################################################################################################
# Chart Methods
##############################################################################################################     
def create_graph_1(user_type):
    global data  

    user_settings = {
        'consumer': {
            'value_column': ['Stromverbrauch [kWh]'],
            'color_map': {'Stromverbrauch [kWh]': 'blue'},
        },
        'prosumer': {
            'value_column': ['Stromverbrauch [kWh]', 'Stromerzeugung [kWh]'],
            'color_map': {'Stromverbrauch [kWh]': 'blue', 'Stromerzeugung [kWh]': 'green'},
        },
        'flexumer': {
            'value_column': ['Stromverbrauch [kWh]', 'Stromerzeugung [kWh]', 'Stromspeicher [kWh]'],
            'color_map': {'Stromverbrauch [kWh]': 'blue', 'Stromerzeugung [kWh]': 'green', 'Stromspeicher [kWh]': 'red'},
        }
    }
    
    settings = user_settings[user_type]
    
    fig_1 = px.line(data,
                    x='Zeitstempel',
                    y=settings['value_column'],
                    title= 'Stromverteilung im Viertelstundentakt',
                    labels={'Zeitstempel': 'Zeitstempel','variable': 'Variable', 'value': 'Strom [kWh]'},
                    color_discrete_map=settings['color_map'])
    
    fig_1.update_layout(legend=dict(title=""))

    avg_values = data[settings['value_column']].mean()

    for col, avg_value in avg_values.items():
        fig_1.add_trace(go.Scatter(
            x=[data['Zeitstempel'].iloc[0], data['Zeitstempel'].iloc[-1]],
            y=[avg_value, avg_value],
            mode='lines',
            name=f"AVG({col}): {avg_value:.2f} kWh/Viertelstunde",
            line=dict(color='black', dash='dash'),
            visible='legendonly'
        ))
    fig_1.update_xaxes(range=[data['Zeitstempel'].min(), data['Zeitstempel'].max()])
    
    return fig_1

def create_graph_2(user_type):
    global data
    
    month_abbr = [calendar.month_abbr[i] for i in range(1, 13)]
    
    monthly_data = data.groupby(data['Zeitstempel'].dt.month).sum()
    monthly_consumption = monthly_data['Stromverbrauch [kWh]']
    
    fig_2 = go.Figure()

    fig_2.add_trace(go.Bar(x=month_abbr,
                           y=monthly_consumption.values,
                           text=monthly_consumption.values.round(2),
                           name='Stromverbrauch [kWh]',
                           marker_color='blue',
                           textposition='auto',
                           textangle=0))
    
    if user_type != 'consumer':
        monthly_production = monthly_data['Stromerzeugung [kWh]']
        fig_2.add_trace(go.Bar(x=month_abbr,
                               y=monthly_production.values,
                               text=monthly_production.values.round(2),
                               name='Stromerzeugung [kWh]',
                               marker_color='green',
                               textposition='auto',
                               textangle=0))
    
    annual_consumption_mean = monthly_consumption.mean()
    fig_2.add_trace(go.Scatter(x=month_abbr,
                               y=[annual_consumption_mean] * 12,
                               mode='lines',
                               name=f'AVG(Stromverbrauch): {annual_consumption_mean:.2f} kWh/Monat',
                               line=dict(color='black', dash='dash', width=2), 
                               visible='legendonly'))
    
    if user_type != 'consumer':
        annual_production_mean = monthly_production.mean()
        fig_2.add_trace(go.Scatter(x=month_abbr,
                                   y=[annual_production_mean] * 12,
                                   mode='lines',
                                   name=f'AVG(Stromerzeugung): {annual_production_mean:.2f} kWh/Monat',
                                   line=dict(color='black', dash='dash', width=2), 
                                   visible='legendonly'))
        
    fig_2.update_layout(
        xaxis_title='Monat', 
        yaxis_title='Strom [kWh]', 
        title='Kumulierte Stromverteilung pro Monat',
        xaxis=dict(tickmode='array', tickvals=month_abbr, ticktext=month_abbr),
        xaxis_title_standoff=25
    )
    return fig_2

def create_graph_3(user_type):
    global data

    user_details = {
        'consumer': {
            'value_column': ['Lasten Netz [kWh]'],
            'color_map': {'Lasten Netz [kWh]': 'blue'}
        },
        'prosumer': {
            'value_column': ['Lasten Netz [kWh]', 'Lasten Stromerzeugung [kWh]'],
            'color_map': {'Lasten Netz [kWh]': 'blue', 'Lasten Stromerzeugung [kWh]': 'green'}
        },
        'flexumer': {
            'value_column': ['Lasten Netz [kWh]', 'Lasten Stromerzeugung [kWh]', 'Lasten Stromspeicher [kWh]'],
            'color_map': {'Lasten Netz [kWh]': 'blue', 'Lasten Stromerzeugung [kWh]': 'green', 'Lasten Stromspeicher [kWh]': 'red'}
        }
    }

    details = user_details[user_type]

    df_graph_3 = data[['Zeitstempel'] + details['value_column']]

    fig_3 = px.line(df_graph_3,
                    x='Zeitstempel',
                    y=details['value_column'],
                    title='Lastprofil im Viertelstundentakt',
                    labels={'variable': 'Variable', 'value': 'Strom [kWh]'},
                    color_discrete_map=details['color_map'])

    fig_3.update_layout(xaxis_title='Zeitstempel', legend=dict(title=""))

    for column in details['value_column']:
        mean_value = df_graph_3[column].mean()
        fig_3.add_trace(go.Scatter(x=[data['Zeitstempel'].iloc[0], data['Zeitstempel'].iloc[-1]],
                                   y=[mean_value, mean_value],
                                   mode='lines',
                                   name=f"AVG({column}): {mean_value:.2f} kWh/Viertelstunde",
                                   line=dict(color='black', dash='dash'),
                                   visible='legendonly'))

    fig_3.update_xaxes(range=[data['Zeitstempel'].min(), data['Zeitstempel'].max()])

    return fig_3

def create_graph_4(user_type):
    global data

    fig_4 = go.Figure()

    months = [calendar.month_abbr[i] for i in range(1, 13)]

    bar_data = [
        {'name': 'Lasten Netz [kWh]', 'color': 'blue'},
        {'name': 'Lasten Stromerzeugung [kWh]', 'color': 'green'},
        {'name': 'Lasten Stromspeicher [kWh]', 'color': 'red'}
    ]

    for bar_info in bar_data:
        bar_name = bar_info['name']
        bar_color = bar_info['color']

        if user_type == 'consumer' and bar_name != 'Lasten Netz [kWh]':
            continue

        if bar_name in data.columns:
            monthly_data = data.groupby(data['Zeitstempel'].dt.month)[bar_name].sum()
            fig_4.add_trace(go.Bar(
                x=months,
                y=monthly_data.values,
                text=monthly_data.round(2),
                name=bar_name,
                marker_color=bar_color
            ))

    fig_4.update_layout(
        barmode='stack',
        xaxis_title='Monat',
        yaxis_title='Strom [kWh]',
        title='Verteilung des monatlichen Stromverbrauchs nach Art der Last'
    )

    return fig_4

def create_graph_5():
    global data
    
    df_graph_5 = data[['Zeitstempel', 'Strommarktpreis [€/kWh]']]

    fig_5 = px.line(df_graph_5,
                    x= 'Zeitstempel',
                    y=['Strommarktpreis [€/kWh]'],
                    title='Strompreisentwicklung 2023 (Day-Ahead-Markt)',
                    labels={'value': 'Strompreis [€/kWh]', 'variable': 'Variable', 'Datum': 'Datum'},
                    color_discrete_map={'Strommarktpreis [€/kWh]': '#9B59B6'})
    
    fig_5.update_layout(legend=dict(title=""))
    
    fig_5.add_trace(go.Scatter(x=[df_graph_5['Zeitstempel'].iloc[0], df_graph_5['Zeitstempel'].iloc[-1]],
                               y=[data['Strommarktpreis [€/kWh]'].mean(), data['Strommarktpreis [€/kWh]'].mean()],
                               mode='lines',
                               name=f"AVG(Strommarktpreis): {data['Strommarktpreis [€/kWh]'].mean():.2f} €/kWh",
                               line=dict(color='black', dash='dash'),
                               visible='legendonly'))
    
    fig_5.update_xaxes(range=[data['Zeitstempel'].min(), data['Zeitstempel'].max()])
    
    return fig_5

def create_graph_6():
    global data
    
    df_graph_6 = data[['Zeitstempel', 'Dynamischer Stromtarif [€/kWh]', 'Fixer Stromtarif [€/kWh]']]

    fig_6 = px.line(df_graph_6,
                    x='Zeitstempel',
                    y=['Dynamischer Stromtarif [€/kWh]', 'Fixer Stromtarif [€/kWh]'],
                    title='Strompreisentwicklung entsprechend dem Lastprofil (dynamischer vs. fixer Stromtarif)',
                    labels={'value': 'Strompreis [€/kWh]', 'variable': 'Variable', 'Zeitstempel': 'Zeitstempel'},
                    color_discrete_map={'Dynamischer Stromtarif [€/kWh]': 'olive', 'Fixer Stromtarif [€/kWh]': 'orange'})
    
    fig_6.update_layout(legend=dict(title=""))
    
    for column in df_graph_6.columns[1:]:
        mean_value = df_graph_6[column].mean()
        fig_6.add_trace(go.Scatter(x=[df_graph_6['Zeitstempel'].iloc[0], df_graph_6['Zeitstempel'].iloc[-1]],
                                   y=[mean_value] * 2,
                                   mode='lines',
                                   name=f"AVG({column}): {mean_value:.2f} €/Viertelstunde",
                                   line=dict(color='black', dash='dash'),
                                   visible='legendonly'))
    
    fig_6.update_xaxes(range=[data['Zeitstempel'].min(), data['Zeitstempel'].max()])

    return fig_6

def create_graph_7():
    global data
    
    monthly_tariffs = data.groupby(data['Zeitstempel'].dt.month)[['Fixer Stromtarif [€/kWh]', 'Dynamischer Stromtarif [€/kWh]']].sum()

    fig_7 = go.Figure()

    for col in monthly_tariffs.columns:
        fig_7.add_trace(go.Bar(x=[calendar.month_abbr[i] for i in range(1, 13)],
                               y=monthly_tariffs[col].values,
                               name=col,
                               marker_color='orange' if col == 'Fixer Stromtarif [€/kWh]' else 'olive',
                               text=monthly_tariffs[col].round(2),
                               textposition='auto',
                               textangle=0))

    fig_7.update_layout(
        barmode='group',
        xaxis_title='Monat',
        yaxis_title='Tarifpreis in €',
        title='Stromkosten entsprechend dem Lastprofil (dynamischer vs. fixer Stromtarif)',
        xaxis=dict(tickangle=0)
    )

    annual_tariff_means = monthly_tariffs.mean()

    for col, mean_value in annual_tariff_means.items():
        fig_7.add_trace(go.Scatter(x=[calendar.month_abbr[i] for i in range(1, 13)],
                                   y=[mean_value] * 12,
                                   mode='lines',
                                   name=f'AVG({col}): {mean_value:.2f} €/Monat',
                                   line=dict(color='black', dash='dash', width=2),
                                   visible='legendonly'))

    return fig_7

def create_graph_8():
    global predictions
    
    df_graph_8 = predictions[['Zeitstempel', 'Pred_Lasten Netz [kWh]']]

    fig_8 = px.line(
        df_graph_8,
        x='Zeitstempel',
        y=['Pred_Lasten Netz [kWh]'],
        title='Prognose des Netzstromverbrauchs in Viertelstundentakt für das Jahr 2024',
        labels={'value': 'Strom [kWh]', 'variable': 'Variable', 'Zeitstempel': 'Zeitstempel'},
        color_discrete_map={'Pred_Lasten Netz [kWh]': '#2DD9FF'}
    )

    fig_8.update_layout(legend=dict(title=""))

    avg_value_8 = df_graph_8['Pred_Lasten Netz [kWh]'].mean()
    fig_8.add_trace(
        go.Scatter(
            x=[df_graph_8['Zeitstempel'].iloc[0], df_graph_8['Zeitstempel'].iloc[-1]],
            y=[avg_value_8, avg_value_8],
            mode='lines',
            name=f"AVG(Pred_Lasten Netz): {avg_value_8:.2f} kWh/Viertelstunde",
            line=dict(color='black', dash='dash'),
            visible='legendonly'
        )
    )

    fig_8.update_xaxes(range=[df_graph_8['Zeitstempel'].min(), df_graph_8['Zeitstempel'].max()])

    return fig_8

def create_graph_9():
    global predictions
    
    df_graph_9 = predictions[['Zeitstempel', 'Pred_Strommarktpreis [€/kWh]']]

    fig_9 = px.line(
        df_graph_9,
        x='Zeitstempel',
        y=['Pred_Strommarktpreis [€/kWh]'],
        title='Strompreisentwicklung 2024 (Day-Ahead-Markt)',
        labels={'value': 'Strompreis [€/kWh]', 'variable': 'Variable', 'Zeitstempel': 'Zeitstempel'},
        color_discrete_map={'Pred_Strommarktpreis [€/kWh]': '#E52CF4'}
    )

    fig_9.update_layout(legend=dict(title=""))

    avg_value_9 = df_graph_9['Pred_Strommarktpreis [€/kWh]'].mean()
    fig_9.add_trace(
        go.Scatter(
            x=[df_graph_9['Zeitstempel'].iloc[0], df_graph_9['Zeitstempel'].iloc[-1]],
            y=[avg_value_9, avg_value_9],
            mode='lines',
            name=f"AVG(Pred_Strommarktpreis): {avg_value_9:.2f} €/kWh",
            line=dict(color='black', dash='dash'),
            visible='legendonly'
        )
    )

    fig_9.update_xaxes(range=[df_graph_9['Zeitstempel'].min(), df_graph_9['Zeitstempel'].max()])

    return fig_9


        
##############################################################################################################
# Callbacks
##############################################################################################################
@app.callback(
    Output('dynamic-image', 'src'),
    [Input('user-type-radio', 'value')]
)
def update_image(user_type):
    return load_image(user_type)

@app.callback(
    [Output('pv-capacity', 'options'),
     Output('pv-capacity', 'value')],
    [Input('user-type-radio', 'value')]
)
def update_pv_capacity_options(user_type):
    if user_type == 'consumer':
        options = [{'label': 'N/A', 'value': 'N/A'}]
        value = 'N/A'
    else:
        options = [
            {'label': f'{i} kWp', 'value': f'PV-{i}kWp'} for i in [1, 5, 10, 15, 20, 25, 30]
        ]
        value = 'PV-1kWp'
    return options, value
    
@app.callback(
    [Output('battery-capacity', 'options'),
     Output('battery-capacity', 'value')],
    [Input('user-type-radio', 'value')]
)
def update_battery_capacity_options(user_type):
    if user_type == 'flexumer':
        options = [
            {'label': f'{cap} kWh', 'value': f'{cap} kWh'} for cap in ['2,5', '5,1', '7,7', '10,2', '12,8']
        ]
        value = '2,5 kWh'
    else:
        options = [{'label': 'N/A', 'value': 'N/A'}]
        value = 'N/A'
    return options, value

@app.callback(
    [Output('simulation-results', 'children'),
     Output('simulation-view', 'style')],
    [Input('simulate-button', 'n_clicks')],
    [State('annual-consumption-input', 'value'),
     State('current-tariff-input', 'value'),
     State('current-base-input', 'value'),
     State('pv-capacity', 'value'),
     State('user-type-radio', 'value'),
     State('battery-capacity', 'value')]
)
def update_simulation_results(n_clicks, annual_input, tariff_input, base_input, pv_capacity, user_type, battery_capacity):
    global data, predictions

    if n_clicks > 0:
        run_simulator(annual_input, tariff_input, base_input, pv_capacity, user_type, battery_capacity)
        predictions = run_machine_learning(data)[['Zeitstempel', 'Stunde', 'Tag', 'Wochentag', 'Tag des Jahres', 'Kalenderwoche', 'Monat', 'Quartal', 'Jahr', 'Pred_Lasten Netz [kWh]', 'Pred_Strommarktpreis [€/kWh]']]
        
        return generate_graphs(user_type), {'display': 'block'}
    else:
        return '', {'display': 'none'}

@app.callback(
    Output('save-button', 'n_clicks'),
    [Input('save-button', 'n_clicks')]
)
def save_to_excel_callback(n_clicks):
    if n_clicks is not None and n_clicks > 0:
        save_to_excel()
    return 0
    
##############################################################################################################
# Starter
##############################################################################################################
def run_simulation():
    global data
    %run "SGS/Simulations-App.ipynb"
    data = load_data()
    app.layout = build_layout()
    app.run_server(debug=True, port=8050)